In [1]:
%matplotlib inline
%run ../../import_envs.py
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 1.1.0 cuda: True


In [2]:
## Load dataset
data_path = "../gmm_dataset_c20k"
Data = torch.from_numpy(np.load(data_path + '/obs.npy')).float()

NUM_DATASETS, N, D = Data.shape
K = 3 ## number of clusters
SAMPLE_SIZE = 10
NUM_HIDDEN_LOCAL = 32

MCMC_SIZE = 10
BATCH_SIZE = 20
NUM_EPOCHS = 350
LEARNING_RATE = 1e-4
CUDA = torch.cuda.is_available()
PATH = 'AG-sym-weights-%dsteps-%dsamples' % (MCMC_SIZE, SAMPLE_SIZE)
DEVICE = torch.device('cuda:1')

Train_Params = (NUM_EPOCHS, NUM_DATASETS, SAMPLE_SIZE, BATCH_SIZE, CUDA, DEVICE, PATH)
Model_Params = (N, K, D, MCMC_SIZE)

In [3]:
from local_enc import *
from global_oneshot import *
from global_enc import *
## if reparameterize continuous variables
Reparameterized = False
# initialization
enc_z = Enc_z(K, D, NUM_HIDDEN_LOCAL, CUDA, DEVICE)
enc_eta = Enc_eta(K, D, CUDA, DEVICE, Reparameterized)
oneshot_eta = Oneshot_eta(K, D, CUDA, DEVICE, Reparameterized)

if CUDA:
    enc_z.cuda().to(DEVICE)
    enc_eta.cuda().to(DEVICE)
    oneshot_eta.cuda().to(DEVICE)
optimizer =  torch.optim.Adam(list(oneshot_eta.parameters())+list(enc_eta.parameters())+list(enc_z.parameters()),lr=LEARNING_RATE, betas=(0.9, 0.99))
models = (oneshot_eta, enc_eta, enc_z)

In [4]:
from ag_ep import *
train(models, EUBO_init_eta, optimizer, Data, Model_Params, Train_Params)

epoch: 0\350 (120s),  symKL_DB_eta: 297.190,  symKL_DB_z: 270.691,  loss: -1380.396,  ess: 3.182,  kl_eta_ex: 747.423,  kl_eta_in: 160.060,  kl_z_ex: 178.864,  kl_z_in: 32.282
epoch: 1\350 (120s),  symKL_DB_eta: 203.402,  symKL_DB_z: 134.395,  loss: -1112.150,  ess: 3.635,  kl_eta_ex: 436.999,  kl_eta_in: 146.446,  kl_z_ex: 80.220,  kl_z_in: 21.403
epoch: 2\350 (120s),  symKL_DB_eta: 147.413,  symKL_DB_z: 85.750,  loss: -973.571,  ess: 3.958,  kl_eta_ex: 302.276,  kl_eta_in: 148.975,  kl_z_ex: 45.872,  kl_z_in: 15.491
epoch: 3\350 (120s),  symKL_DB_eta: 111.119,  symKL_DB_z: 60.631,  loss: -896.947,  ess: 4.242,  kl_eta_ex: 229.183,  kl_eta_in: 160.604,  kl_z_ex: 29.187,  kl_z_in: 11.617
epoch: 4\350 (120s),  symKL_DB_eta: 89.964,  symKL_DB_z: 48.134,  loss: -850.020,  ess: 4.452,  kl_eta_ex: 187.918,  kl_eta_in: 173.953,  kl_z_ex: 21.474,  kl_z_in: 9.317
epoch: 5\350 (120s),  symKL_DB_eta: 77.899,  symKL_DB_z: 41.887,  loss: -815.208,  ess: 4.575,  kl_eta_ex: 162.600,  kl_eta_in: 184.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/hao/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3296, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-d8a561a5e9a6>", line 2, in <module>
    train(models, EUBO_init_eta, optimizer, Data, Model_Params, Train_Params)
  File "../training.py", line 29, in train
    loss, metric_step, reused = objective(models, obs, SubTrain_Params)
  File "../objectives/ag_ep.py", line 39, in EUBO_init_eta
    q_eta, p_eta, q_nu = enc_eta(obs, state, K, D)
  File "/home/hao/anaconda3/lib/python3.7/site-packages/torch/nn/modules/module.py", line 493, in __call__
    result = self.forward(*input, **kwargs)
  File "../models/global_enc.py", line 61, in forward
    name='precisions')
  File "<string>", line 2, in f
  File "/home/hao/Research/probtorch/probtorch/stochastic.py", line 287, in variable
    node = RandomVariable(dist, value, provenance, mask=self._mask)
  File "/home/hao/Re

KeyboardInterrupt: 

In [ ]:
torch.save(enc_z.state_dict(), "../weights/enc-z-%s" % PATH)
torch.save(enc_eta.state_dict(), "../weights/enc-eta-%s" % PATH)
torch.save(oneshot_eta.state_dict(), "../weights/oneshot-eta-%s" % PATH)